# SELECT
다양한 조건을 사용하여 SELECT문을 사용해본다

In [1]:
import pymysql
import pandas as pd
import numpy as np
from config import *
import uuid
import datetime 
from itertools import product

In [2]:
def query(sql, data):
    """
    DB에 SQL문 실행(query: 데이터베이스에 정보 요청)
       Args:
           sql(str): sql 구문
           data(list or tuple): sql 파라미터. ex) [val1, val2 ...]
       Returns:
           query 결과. 결과가 없으면 빈 tuple 리턴
    """
    # connect(): DB와 연결
    db = pymysql.Connect(host=MYSQL_HOST, 
                     user=MYSQL_USER, 
                     password=MYSQL_PASSWORD, 
                     database=MYSQL_DATABASE,
                     autocommit=MYSQL_AUTOCOMMIT, # Query 실행 후 자동 commit(실제 insert, update 반영)
                     charset=MYSQL_CHARSET, # 인코딩 정보
                     cursorclass=pymysql.cursors.DictCursor # DB를 조회한 결과를 Column 명이 Key인 Dictionary로 저장
                    )
    cursor = db.cursor()

    cursor.execute(sql, data) 
    result = cursor.fetchall() # fetchall(): 모든 데이터를 한 번에 가져옴
    db.close()
    return to_pandas(result)

def to_pandas(result):
    return pd.DataFrame(result)

## 기본 SELECT

In [3]:
# 테이블의 전체 데이터를 select
sql = "SELECT * FROM " + MYSQL_TABLENAME+ ";"
print(sql)
result = query(sql, ())
result.head()

SELECT * FROM mytable;


,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01
0,1,b177b428b46311edbb8940b076a1ec6b,2023-02-24 16:52:59,Choi,Choi@mysql.com,010-9876-5432,35,"[{""search"": ""naver""}, {""viewing time"": ""30""}]",,
1,2,b1778d08b46311edae6040b076a1ec6b,2023-02-24 16:52:59,Lee,Lee@mysql.com,010-4567-8910,80,"[{""search"": ""youtube""}, {""viewing time"": ""54""}]",,
2,3,b1773ed9b46311edaa0240b076a1ec6b,2023-02-24 16:52:59,Kim,Kim@mysql.com,010-1234-5678,44,"[{""search"": ""github""}, {""viewing time"": ""64""}]",,
3,4,b177b429b46311edbf3540b076a1ec6b,2023-02-24 15:52:59,Choi,Choi@mysql.com,010-9876-5432,29,"[{""search"": ""github""}, {""viewing time"": ""33""}]",,
4,5,b1778d09b46311ed8d5f40b076a1ec6b,2023-02-24 15:52:59,Lee,Lee@mysql.com,010-4567-8910,0,"[{""search"": ""naver""}, {""viewing time"": ""19""}]",,


In [4]:
result.info() # 총 144rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   index     144 non-null    int64         
 1   uuid      144 non-null    object        
 2   reg_date  144 non-null    datetime64[ns]
 3   name      144 non-null    object        
 4   email     144 non-null    object        
 5   phone     144 non-null    object        
 6   energy    144 non-null    int64         
 7   label     144 non-null    object        
 8   mark      144 non-null    object        
 9   ext01     144 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 11.4+ KB


## SELECT + 자주쓰는 기본 문법
- DATE, COALESCE

In [5]:
sql = "SELECT DATE('2023-02-24 11:00:00');" # date 생성해서 테이블처럼 만듦
print(sql)
result = query(sql, ())
result.head()

SELECT DATE('2023-02-24 11:00:00');


,DATE('2023-02-24 11:00:00')
0,2023-02-24


In [6]:
sql = "SELECT COALESCE('2023-02-24 11:00:00', 0);" # date 생성해서 테이블처럼 만듦
print(sql)
result = query(sql, ())
result.head()

SELECT COALESCE('2023-02-24 11:00:00', 0);


,"COALESCE('2023-02-24 11:00:00', 0)"
0,2023-02-24 11:00:00


In [7]:
sql = "SELECT COALESCE(null, 10, 5);" # 값이 null이 아니면서 가장 먼저 입력한 10이 나옴. null만 있다면 null만 나옴
print(sql)
result = query(sql, ())
result.head()

SELECT COALESCE(null, 10, 5);


,"COALESCE(null, 10, 5)"
0,10


## SELECT + WHERE + 조건

In [8]:
# name 지정
sql = "SELECT * FROM " + MYSQL_TABLENAME +" where name=%s;" 
print(sql)
result = query(sql, (['Kim']))
result[['name']].value_counts()

SELECT * FROM mytable where name=%s;


name
Kim     48
dtype: int64

In [10]:
# 시간 지정
sql = "SELECT * FROM " + MYSQL_TABLENAME +" where reg_date BETWEEN TIMESTAMP(%s) AND TIMESTAMP(%s);" 
print(sql)
result = query(sql, (['2023-02-24 00:00:00', '2023-02-25 00:00:00']))
result.sort_values('reg_date')

SELECT * FROM mytable where reg_date BETWEEN TIMESTAMP(%s) AND TIMESTAMP(%s);


,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01
50,51,b17765f9b46311ed83b740b076a1ec6b,2023-02-24 00:52:59,Kim,Kim@mysql.com,010-1234-5678,35,"[{""search"": ""naver""}, {""viewing time"": ""75""}]",,
49,50,b1778d18b46311edac5340b076a1ec6b,2023-02-24 00:52:59,Lee,Lee@mysql.com,010-4567-8910,83,"[{""search"": ""github""}, {""viewing time"": ""32""}]",,
48,49,b177db27b46311edbe2b40b076a1ec6b,2023-02-24 00:52:59,Choi,Choi@mysql.com,010-9876-5432,16,"[{""search"": ""naver""}, {""viewing time"": ""36""}]",,
47,48,b17765f8b46311ed805240b076a1ec6b,2023-02-24 01:52:59,Kim,Kim@mysql.com,010-1234-5678,74,"[{""search"": ""youtube""}, {""viewing time"": ""23""}]",,
46,47,b1778d17b46311ed923840b076a1ec6b,2023-02-24 01:52:59,Lee,Lee@mysql.com,010-4567-8910,80,"[{""search"": ""youtube""}, {""viewing time"": ""61""}]",,
45,46,b177db26b46311edaf7c40b076a1ec6b,2023-02-24 01:52:59,Choi,Choi@mysql.com,010-9876-5432,0,"[{""search"": ""naver""}, {""viewing time"": ""63""}]",,
44,45,b17765f7b46311ed88a840b076a1ec6b,2023-02-24 02:52:59,Kim,Kim@mysql.com,010-1234-5678,57,"[{""search"": ""youtube""}, {""viewing time"": ""31""}]",,
43,44,b1778d16b46311ed91c040b076a1ec6b,2023-02-24 02:52:59,Lee,Lee@mysql.com,010-4567-8910,29,"[{""search"": ""github""}, {""viewing time"": ""25""}]",,
42,43,b177db25b46311edb23240b076a1ec6b,2023-02-24 02:52:59,Choi,Choi@mysql.com,010-9876-5432,81,"[{""search"": ""github""}, {""viewing time"": ""58""}]",,
41,42,b17765f6b46311ed899140b076a1ec6b,2023-02-24 03:52:59,Kim,Kim@mysql.com,010-1234-5678,32,"[{""search"": ""github""}, {""viewing time"": ""9""}]",,


In [ ]:
# name 지정 + 시간 지정
sql = "SELECT * FROM " + MYSQL_TABLENAME +" where name=%s AND reg_date BETWEEN TIMESTAMP(%s) AND TIMESTAMP(%s);" 
print(sql)
result = query(sql, (['Kim', '2023-02-12 00:00:00', '2023-02-13 00:00:00']))
result.sort_values('reg_date')

## SELECT + LIMIT
- 조회 개수 제한
- 참고로 String Format을 사용하면 d %s %f 등 지정해주는 것이 맞지만 pymysql에서 변수 타입과 상관없이 전부 %s를 사용해줘야 한다

In [ ]:
# name 지정
sql = "SELECT * FROM " + MYSQL_TABLENAME +" where name=%s LIMIT %s;" # 변수 타입 상관없이 %s
print(sql)
result = query(sql, (['Kim', 5])) 
result[['name']].value_counts()

In [ ]:
result

## SELECT + ORDER BY + LIMIT N,M
- 오름차순으로 내림/오름 차순으로 정렬 뒤에 N행 부터 M개 행을 가져옴
    - LIMITE N, M: N행에서부터 M개를 가져옴

In [ ]:
# LIMIT로 지정한 행을 가져올 수 있음 
sql = "SELECT * FROM " + MYSQL_TABLENAME +" where name=%s ORDER BY reg_date DESC  LIMIT %s, %s;" # 내림차순
print(sql)
result = query(sql, (['Kim', 2, 5])) 
result

## SELECT + COUNT

In [ ]:
# row counts
sql = "SELECT COUNT(*) FROM " + MYSQL_TABLENAME +";" # 내림차순
print(sql)
result = query(sql, ()) 
result

In [ ]:
# row counts + 조건문
sql = "SELECT COUNT(*) FROM " + MYSQL_TABLENAME +" where name=%s;" # 내림차순
print(sql)
result = query(sql, (['Kim'])) 
result

## SELECT + LIKE
- 특정 문자열이 있는지 검색 LIKE에 붙는 '%'위치에 따라 조건이 다름

In [ ]:
# "[{'serach':%": [{'serach':로 시작하는 데이터 검색
sql = "SELECT * FROM " + MYSQL_TABLENAME + " where label LIKE %s;"
print(sql)
result = query(sql, ([ "[{'serach':%" ])) 
result

In [ ]:
# "%64}]": 64}]로 끝나는 데이터 검색
sql = "SELECT * FROM " + MYSQL_TABLENAME + " where label LIKE %s;"
print(sql)
result = query(sql, ([ "%64}]" ])) 
result

In [ ]:
# "%[{'serach': 'youtube'}%": [{'serach': 'youtube'}이 포함되는 데이터 검색
sql = "SELECT * FROM " + MYSQL_TABLENAME + " where label LIKE %s;"
print(sql)
result = query(sql, ([ "%[{'serach': 'youtube'}%" ])) 
result

## JSON_EXTRACT
- Text가 JSON 방식이면 JSON형태로 조회

In [ ]:
sql = "SELECT JSON_EXTRACT(`label`, '$[*]') FROM " + MYSQL_TABLENAME + " LIMIT 0, 5;"
print(sql)
result = query(sql, ()) 
result

In [ ]:
# 전체에서 .search의 value만 get
sql = "SELECT JSON_EXTRACT(`label`, '$[*].search') FROM " + MYSQL_TABLENAME + " LIMIT 0, 5;"
print(sql)
result = query(sql, ()) 
result

In [ ]:
# JSON_EXTRACT는 ""큰 따옴표를 사용
# 위 array 형태를 $[0]과 같이 정해진 순서 array를 가져올 수 잇음
sql = "SELECT JSON_EXTRACT(`label`, '$[1]') FROM " + MYSQL_TABLENAME + " LIMIT 0, 5;"
print(sql)
result = query(sql, ()) 
result

In [ ]:
# array 로부터 json 형식으로 extract 한 번더 수행
sql = "SELECT JSON_EXTRACT(JSON_EXTRACT(`label`, '$[1]'), '$.viewing time') FROM " + MYSQL_TABLENAME + " LIMIT 0, 5;"
print(sql)
result = query(sql, ()) 
result

## JSON_SEARCH
- JSON_SEARCH(json_doc, 'one'or'all', search str): str을 포함한 하나 혹은 다수의 JSON object 추출

In [ ]:
# array 로부터 json 형식으로 extract 한 번더 수행
sql = "SELECT JSON_SEARCH(JSON_EXTRACT(`label`, '$[*]'), 'one', 'github') FROM " + MYSQL_TABLENAME + " LIMIT 0, 5;"
print(sql)
result = query(sql, ()) 
result

## SUBSTRING
- SUBSTRING(문자열 FROM N for M): N부터 읽고 M개 문자를 읽어옴

In [ ]:
# array 로부터 json 형식으로 extract 한 번더 수행
sql = "SELECT SUBSTRING(label FROM 4 for 6) FROM " + MYSQL_TABLENAME + " LIMIT 0, 5;"
print(sql)
result = query(sql, ()) 
result

## JSON_SEARCH + SUBSTRING
- JSON_SEARCH + SUBSTRING 두 가지를 사용하여 Object index를 알 수 있음

In [ ]:
# array 로부터 json 형식으로 extract 한 번더 수행
sql = "SELECT SUBSTRING(JSON_SEARCH(JSON_EXTRACT(`label`, '$[*]'), 'one', 'github') from 4 for 1) FROM " + MYSQL_TABLENAME + " LIMIT 0, 5;"
print(sql)
result = query(sql, ()) 
result

## GROUP BY
- 그룹화하여 데이터 조회. 단, 그룹화 한 column을 그대로 select에 사용해야함

## INNER